In [149]:
from pathlib import Path
import shutil
import os
import math

In [150]:
OUTPUT_DATASET_PATH     =    r"E:\datasets"
INPUT_IMG_PATHs         =   [r"E:\Bags\red\cart_1_red\png\color\images", 
                             r"E:\Bags\red\cart_2_red\png\color\images", 
                             r"E:\Bags\red\cart_3_red\png\color\images",
                             r"E:\Bags\yellow\cart_1_yellow\png\color\images",
                             r"E:\Bags\yellow\cart_2_yellow\png\color\images",
                             r"E:\Bags\yellow\cart_3_red_yellow\png\color\images",
                             r"E:\Bags\yellow\cart_4_yellow_red\png\color\images"]                    

DATASET_SIZE            = 600
SAME_NUMBER_IMGS        = True


In [151]:
images_per_path = math.floor(DATASET_SIZE / len(INPUT_IMG_PATHs))
print("Final dataset size: \t {} \t Location: \t{}".format(DATASET_SIZE, OUTPUT_DATASET_PATH))
print("Paths given: \t{} \t Images copied from each path: \t {} \n".format(len(INPUT_IMG_PATHs), images_per_path))

total_files         = 0
total_real_files    = 0
saving_jump         = list()
real_saved          = list()

for path in INPUT_IMG_PATHs:
    files_number        = len([name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))])
    saving_jump.append(int(math.floor(files_number / images_per_path)))
    real_saved.append(math.floor(files_number / saving_jump[-1]))
    recording_name      = path.split('\\')[3]

    total_files         = total_files + files_number
    total_real_files    = total_real_files + real_saved[-1]

    print("Total images: {:,}  \t saving every {} \t saved images: {} \t [{}]".format(files_number, saving_jump[-1], real_saved[-1], recording_name))



print("\nTotal images: {:,} (paths) \t Total images saved: {:,}".format(total_files, total_real_files))

Final dataset size: 	 600 	 Location: 	E:\dataset
Paths given: 	7 	 Images copied from each path: 	 85 

Total images: 1,657  	 saving every 19 	 saved images: 87 	 [cart_1_red]
Total images: 1,177  	 saving every 13 	 saved images: 90 	 [cart_2_red]
Total images: 1,290  	 saving every 15 	 saved images: 86 	 [cart_3_red]
Total images: 693  	 saving every 8 	 saved images: 86 	 [cart_1_yellow]
Total images: 663  	 saving every 7 	 saved images: 94 	 [cart_2_yellow]
Total images: 5,401  	 saving every 63 	 saved images: 85 	 [cart_3_red_yellow]
Total images: 5,258  	 saving every 61 	 saved images: 86 	 [cart_4_yellow_red]

Total images: 16,139 (paths) 	 Total images saved: 614


In [152]:


for path, jump, real_saved  in zip(INPUT_IMG_PATHs, saving_jump, real_saved):
    total_img_path  = 0
    saved_img_num   = 0
    print("Path: {} \t images to be saved: {}".format(path, real_saved))
    for image in Path(path).rglob('*.png'):
        if total_img_path % jump == 0:
            shutil.copy(image, Path(OUTPUT_DATASET_PATH))
            #print("[{}] saved image {} ".format(path.split('\\')[3], saved_img_num))
            saved_img_num = saved_img_num + 1
        total_img_path = total_img_path + 1
    print("Done")

        
    

Path: E:\Bags\red\cart_1_red\png\color\images 	 images to be saved: 87
Done
Path: E:\Bags\red\cart_2_red\png\color\images 	 images to be saved: 90
Done
Path: E:\Bags\red\cart_3_red\png\color\images 	 images to be saved: 86
Done
Path: E:\Bags\yellow\cart_1_yellow\png\color\images 	 images to be saved: 86
Done
Path: E:\Bags\yellow\cart_2_yellow\png\color\images 	 images to be saved: 94
Done
Path: E:\Bags\yellow\cart_3_red_yellow\png\color\images 	 images to be saved: 85
Done
Path: E:\Bags\yellow\cart_4_yellow_red\png\color\images 	 images to be saved: 86
Done


## Move images in json

In [126]:
import sys
import os, random
import shutil
import json

random.seed(10)

In [127]:
# Input arguments
arg_percentage      = 20
arg_input_json      = r'labels\dataset_620_red_yellow_cart_only.json'
arg_input_dir       = r'G:\datasets\dataset_620_red_yellow_cart_only'
arg_output_dir      = r'G:\datasets\620_red_yellow_cart_only'

In [128]:
# Create output folders 
train_dir   = arg_output_dir + r'\train'
val_dir     = arg_output_dir + r'\val'
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

In [129]:
# Calculate total of pictures according to total number of pictures
total_imgs_list     = os.listdir(arg_input_dir)
total_val_imgs      = round(len(imgs_list) * (arg_percentage/100))
# Choose random sample of total_val_imgs length
val_imgs_list       = random.sample(total_imgs_list, total_val_imgs)
train_imgs_list     = []

In [130]:
# Move chosen images to output_dir\val\      
for i in range(total_val_imgs):
    total_imgs_list.remove(val_imgs_list[i])
    shutil.copy(arg_input_dir + "\\" + val_imgs_list[i], val_dir + "\\" + val_imgs_list[i])
train_imgs_list = total_imgs_list
# Move remaining images to output_dir\train\
for i in range(len(train_imgs_list)):
    shutil.copy(arg_input_dir + "\\" + train_imgs_list[i], train_dir + "\\" + train_imgs_list[i])

# Json  Creation

In [135]:
train_json  = {}
val_json    = {}
# Open Json with all images
with open(arg_input_json) as json_file:
    data = json.load(json_file)
# Extract image metadata
img_metadata = data['_via_img_metadata']

In [136]:
# Move val imgs attributes to val_json
for i in range(total_val_imgs):
    # Find json key name. This is needed as the file name has extra numbers
    res = dict(filter(lambda item: val_imgs_list[i] in item[0], img_metadata.items()))
    val_json[list(res.keys())[0]] = img_metadata[list(res.keys())[0]]
# Move train imgs attributes to train_json
for i in range(len(train_imgs_list)):
    # Find json key name:
    res = dict(filter(lambda item: train_imgs_list[i] in item[0], img_metadata.items()))
    train_json[list(res.keys())[0]] = img_metadata[list(res.keys())[0]]
    

In [137]:
# Write attributes to output jsons
train_json_file = train_dir + r'\via_region_data.json'
val_json_file = val_dir + r'\via_region_data.json'

with open(val_json_file, 'w') as outfile:
    json.dump(val_json, outfile)
with open(train_json_file, 'w') as outfile:
    json.dump(train_json, outfile)